In [123]:
import pandas as pd
import numpy as np
import pretty_midi
import music21

In [2]:
import os
os.chdir("/Users/desireewaugh/Desktop/MIT/Courses/6.883 - Modeling with ML/Projects/Final Project/MusicGenerator/data")

## Play abc file

In [13]:
# Run "pip install music21" in your chosen directory
from music21 import *

In [4]:
music_file = converter.parse('rockethornpipe-1.abc')

In [5]:
# Need to download pygame to play files: "pip install pygame"
sp = midi.realtime.StreamPlayer(music_file)
sp.play()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


## With midi files

In [6]:
# Data from: https://magenta.tensorflow.org/datasets/groove#dataset
# Possible dataset: https://drive.google.com/file/d/14e0MCJD7RH_m7CpsFZWPIpO0WgQrwi64/view
os.chdir("./groove/drummer1/eval_session")

In [216]:
midi = pretty_midi.PrettyMIDI("hotel_california.mid")
piano_midi = midi.instruments[0]
piano_roll = piano_midi.get_piano_roll()


piano_roll.shape

(128, 2672)

In [218]:
midi.instruments[0].notes

[Note(start=1.781493, end=1.789610, pitch=36, velocity=103),
 Note(start=1.781493, end=1.789610, pitch=42, velocity=72),
 Note(start=1.781493, end=1.789610, pitch=57, velocity=84),
 Note(start=2.171103, end=2.179220, pitch=44, velocity=101),
 Note(start=2.365908, end=2.374025, pitch=42, velocity=89),
 Note(start=2.560713, end=2.568830, pitch=40, velocity=102),
 Note(start=2.560713, end=2.568830, pitch=42, velocity=45),
 Note(start=2.950323, end=2.958440, pitch=44, velocity=95),
 Note(start=3.145128, end=3.153245, pitch=42, velocity=83),
 Note(start=3.339933, end=3.348050, pitch=36, velocity=103),
 Note(start=3.339933, end=3.348050, pitch=42, velocity=39),
 Note(start=3.729543, end=3.737660, pitch=44, velocity=95),
 Note(start=3.924348, end=3.932465, pitch=42, velocity=89),
 Note(start=4.119153, end=4.127270, pitch=40, velocity=102),
 Note(start=4.119153, end=4.127270, pitch=41, velocity=80),
 Note(start=4.119153, end=4.127270, pitch=42, velocity=45),
 Note(start=4.508763, end=4.516880,

In [196]:
piano_midi

Instrument(program=0, is_drum=True, name="(C) HitTrax: Hotel California")

In [193]:
sum(piano_roll[100,:])

0.0

In [179]:
# Convert midi file to a dataframe
# Reference: https://www.audiolabs-erlangen.de/resources/MIR/FMP/C1/C1S2_MIDI.html
def midi_df(midi_file):
    midi_info = []
    for instrument in midi_file.instruments:
        for note in instrument.notes:
            start = note.start
            end = note.end
            pitch = note.pitch
            velocity = note.velocity
            midi_info.append([start, end, pitch, velocity, 
                              instrument.program, instrument.is_drum, instrument.name])

    df = pd.DataFrame(midi_info, columns=['Start', 'End', 'Pitch', 'Velocity', 
                                                   'Program', 'is_drum', 'Name'])
    return df
    
test_df = midi_df(midi)
test_df.head()

,Start,End,Pitch,Velocity,Program,is_drum,Name
0,1.781493,1.789610,36,103,0,True,(C) HitTrax: Hotel California
1,1.781493,1.789610,42,72,0,True,(C) HitTrax: Hotel California
2,1.781493,1.789610,57,84,0,True,(C) HitTrax: Hotel California
3,2.171103,2.179220,44,101,0,True,(C) HitTrax: Hotel California
4,2.365908,2.374025,42,89,0,True,(C) HitTrax: Hotel California


In [180]:
# Create midi file from a dataframe

# Source: http://craffel.github.io/pretty-midi/

created_midi = pretty_midi.PrettyMIDI()

# Add instruments
for i in range((test_df).shape[0]):
    instrument = pretty_midi.Instrument(program=test_df.iloc[i,4], 
                                        is_drum=test_df.iloc[i,5],
                                        name="Acoustic Grand Piano")

    note = pretty_midi.Note(
        velocity=test_df.iloc[i, 3], 
        pitch=test_df.iloc[i, 2], 
        start=test_df.iloc[i, 0], 
        end=test_df.iloc[i, 1])
    instrument.notes.append(note)
    created_midi.instruments.append(instrument)

created_midi.write("test_midi.midi")
#created_midi.key_signature_changes = midi.key_signature_changes
#created_midi.time_signature_changes = midi.time_signature_changes

midi_matrix(created_midi).head()

,Start,End,Pitch,Velocity,Program,is_drum,Name
0,1.781493,1.789610,36,103,0,True,Acoustic Grand Piano
1,1.781493,1.789610,42,72,0,True,Acoustic Grand Piano
2,1.781493,1.789610,57,84,0,True,Acoustic Grand Piano
3,2.171103,2.179220,44,101,0,True,Acoustic Grand Piano
4,2.365908,2.374025,42,89,0,True,Acoustic Grand Piano


In [181]:
# Play written file
b = converter.parse("test_midi.midi")
sp = music21.midi.realtime.StreamPlayer(b)
sp.play()

In [182]:
# Play actual file
b = converter.parse("hotel_california.mid")
sp = music21.midi.realtime.StreamPlayer(b)
sp.play()